<!-- filepath: c:\Users\Henrique\Desktop\Nova pasta\dashboard\notebooks\depurar_importacao.ipynb -->
<VSCode.Cell language="markdown">
# Configurar Caminhos e Dependências
Defina o caminho absoluto para o arquivo fornecido, valide se ele existe e carregue bibliotecas essenciais (pandas) com ajustes de exibição para facilitar a inspeção dentro do VS Code.
</VSCode.Cell>
<VSCode.Cell language="python">
from pathlib import Path
import importlib
import sys

DATA_PATH = Path(r"c:\Users\Henrique\Downloads\Leads - NUMERADO.csv (1).json")
if not DATA_PATH.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {DATA_PATH}")

try:
    import pandas as pd
except ImportError as exc:
    raise ImportError("Instale pandas com 'pip install pandas' antes de continuar.") from exc

print(f"Python {sys.version.split()[0]} | pandas {pd.__version__}")
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 160)
pd.set_option("display.max_colwidth", 120)
</VSCode.Cell>
<VSCode.Cell language="markdown">
# Validar Encoding e Tamanho do Arquivo
Detecte automaticamente o encoding com chardet, calcule o tamanho em MB e informe se o preview integrado pode estar bloqueado por limite de tamanho/encoding incorreto.
</VSCode.Cell>
<VSCode.Cell language="python">
try:
    import chardet
except ImportError as exc:
    raise ImportError("Instale chardet com 'pip install chardet' antes de continuar.") from exc

with DATA_PATH.open("rb") as fh:
    sample_bytes = fh.read(1_000_000)  # ~1 MB deve ser suficiente para inferência

encoding_info = chardet.detect(sample_bytes)
DETECTED_ENCODING = encoding_info.get("encoding") or "utf-8"
confidence = encoding_info.get("confidence", 0.0)
size_mb = DATA_PATH.stat().st_size / (1024 ** 2)

print(f"Encoding detectado: {DETECTED_ENCODING} (confiança {confidence:.2%})")
print(f"Tamanho do arquivo: {size_mb:.2f} MB")
if size_mb &gt; 50:
    print("Aviso: arquivos acima de ~50 MB podem falhar no preview do VS Code.")
</VSCode.Cell>
<VSCode.Cell language="markdown">
# Inspecionar Estrutura JSON Bruta
Abra o arquivo como JSON puro, confirme que é uma lista de objetos e conte quantos campos nulos existem nos primeiros registros antes de seguir para o pandas.
</VSCode.Cell>
<VSCode.Cell language="python">
import json

with DATA_PATH.open("r", encoding=DETECTED_ENCODING) as fh:
    RAW_DATA = json.load(fh)

if not isinstance(RAW_DATA, list):
    raise TypeError("O arquivo não contém uma lista JSON; verifique o formato exportado.")

record_count = len(RAW_DATA)
sample_size = 5
sample_records = RAW_DATA[:sample_size]
null_overview = []
for idx, record in enumerate(sample_records, start=1):
    total_fields = len(record)
    null_fields = sum(value is None for value in record.values())
    null_overview.append({"registro": idx, "campos": total_fields, "nulos": null_fields})

print(f"Total de registros: {record_count}")
print("Resumo de nulos nos primeiros registros:")
for item in null_overview:
    print(item)

sample_records
</VSCode.Cell>
<VSCode.Cell language="markdown">
# Normalizar Registros em DataFrame
Transforme a lista de objetos em um DataFrame com `json_normalize`, padronize os nomes das colunas e mantenha a referência global `df` para as próximas etapas.
</VSCode.Cell>
<VSCode.Cell language="python">
df_raw = pd.json_normalize(RAW_DATA)
df = df_raw.copy()
df.columns = [col.strip() for col in df.columns]

print(f"DataFrame criado com {df.shape[0]} linhas e {df.shape[1]} colunas")
df.head(3)
</VSCode.Cell>
<VSCode.Cell language="markdown">
# Diagnosticar Colunas Vazias e Tipos
Calcule o percentual de valores nulos por coluna, identifique colunas constantes e normalize tipos importantes (ex.: idade como inteiro).
</VSCode.Cell>
<VSCode.Cell language="python">
null_pct = df.isna().mean().sort_values(ascending=False)
null_report = null_pct.to_frame(name="pct_nulos")
const_columns = [col for col in df.columns if df[col].nunique(dropna=True) &lt;= 1]

if "idade" in df.columns:
    df["idade"] = pd.to_numeric(df["idade"], errors="coerce").astype("Int64")

print("Top 10 colunas com mais nulos:")
display(null_report.head(10))
print("Colunas constantes (sem variação):", const_columns)
</VSCode.Cell>
<VSCode.Cell language="markdown">
# Gerar Prévia Interativa no Notebook
Use o Styler do pandas para confirmar visualmente que a prévia aparece no painel de saída do VS Code.
</VSCode.Cell>
<VSCode.Cell language="python">
from IPython.display import display

preview_rows = df.head(20)
styled_preview = (
    preview_rows.style
    .set_caption("Prévia dos primeiros 20 registros")
    .set_table_styles([{
        "selector": "caption",
        "props": [("caption-side", "top"), ("font-weight", "bold")]
    }])
)
display(styled_preview)
</VSCode.Cell>
<VSCode.Cell language="markdown">
# Exportar Versões Limpas para Pré-visualização no VS Code
Salve o DataFrame tratado em formatos leves (CSV e, se possível, Parquet) dentro da pasta `converted` ao lado do arquivo original e abra-os pelo preview nativo do editor.
</VSCode.Cell>
<VSCode.Cell language="python">
OUTPUT_DIR = DATA_PATH.parent / "converted"
OUTPUT_DIR.mkdir(exist_ok=True)

csv_path = OUTPUT_DIR / "leads_preview.csv"
parquet_path = OUTPUT_DIR / "leads_preview.parquet"

df.to_csv(csv_path, index=False)

try:
    df.to_parquet(parquet_path, index=False)
    parquet_status = f"Parquet gerado em: {parquet_path}"
except Exception as exc:
    parquet_status = f"Falha ao gerar Parquet (instale pyarrow): {exc}"

print(f"CSV salvo em: {csv_path}")
print(parquet_status)
print("Abra o arquivo convertido no VS Code (Open With &gt; CSV/Parquet Viewer) para validar o preview.")
</VSCode.Cell>